In [1]:
import os
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import torch
import sys
import os
import torch
import numpy
import importlib
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict

from sklearn import manifold
from torchsummary import summary
import time

from thop import profile

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [4]:
sys.path.append('/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_dist')
sys.path.append('/workspace/GREAT_ASV_system/Model_exp')

In [5]:
model_path = '/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_logs_201120/xvector(vox2)/model/model000000134.model'
# model_path = '/nvme1/zhiyong/ASV_LOGS_202102/train_logs_201120/xvector(vox2)/model/model000000134.model'

# X-vector

In [6]:
SpeakerNetModel = importlib.import_module('models.'+'X_vector').__getattribute__('MainModel')

In [7]:
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

X_vector.py, Embedding size is 192,  Spec_aug False.


In [8]:
loaded_state = torch.load(model_path, map_location="cuda:0")

In [9]:
self_state = S.state_dict()

for name, param in loaded_state['model'].items():
    origname = name

    ## pass spk clf weight
    if '__L__' in name:
        print('pass __L__ classerfier W')
        continue

    ## pass DA weight
    if 'DA_module' in name:
        print('pass DA_module params:'+name)
        continue

    if name not in self_state:
        name = name.replace("__S__.", "")

        if name not in self_state:
            print("#%s is not in the model."%origname)
            continue

    if self_state[name].size() != loaded_state['model'][origname].size():
        print("#Wrong parameter length: %s, model: %s, loaded: %s"%(origname, self_state[name].size(), loaded_state['model'][origname].size()))
        continue

    self_state[name].copy_(param)

pass __L__ classerfier W


In [10]:
S.cuda()

Xvector_1L(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (tdnn1): Sequential(
    (0): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn2): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn3): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn4): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=

In [11]:
S.eval()

Xvector_1L(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (tdnn1): Sequential(
    (0): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn2): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn3): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn4): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=

In [12]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [13]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [14]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [15]:
len(setfiles)

4708

In [18]:
testcount = 1
starttime = time.time()
for count, wavline in enumerate(setfiles[:testcount]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime))
print(testcount / (endtime-starttime))

0.00654149055480957
152.87035754637898


In [21]:
ref_feat.shape

(1, 192)

In [19]:
ref_feat

array([[ 1.59025699e-01, -2.55970925e-01, -1.84751838e-01,
         2.04936275e-03, -3.68359476e-01, -7.72498176e-02,
         1.18553005e-01, -2.29681715e-01,  2.60705262e-01,
        -1.39664844e-01, -4.00144219e-01, -5.78067660e-01,
        -2.61265486e-01, -2.01939493e-01, -3.04551840e-01,
        -4.08011712e-02, -9.67582986e-02,  2.56850183e-01,
        -4.89437103e-01,  2.10375398e-01,  6.70573041e-02,
         2.43994966e-03,  1.95706248e-01, -2.71901220e-01,
        -3.33187692e-02,  3.20731491e-01, -2.85164863e-01,
         9.59827304e-02, -9.19954106e-02,  5.98023795e-02,
        -1.71101794e-01,  1.06167681e-04, -2.00814784e-01,
         6.85920417e-02, -3.47751379e-02, -1.01280205e-01,
        -1.64571288e-03, -6.00561984e-02,  1.44580543e-01,
         3.14291596e-01, -8.29275418e-03,  1.74917027e-01,
        -1.70172229e-01,  1.32377326e-01, -2.27514446e-01,
        -2.29751877e-02,  1.58675700e-01, -1.36593394e-02,
        -1.21462695e-01, -4.93786251e-03,  6.92019016e-0